In [2]:
# Intialization
import os
import sys

os.environ["SPARK_HOME"] = "/home/talentum/spark"
os.environ["PYLIB"] = os.environ["SPARK_HOME"] + "/python/lib"
# In below two lines, use /usr/bin/python2.7 if you want to use Python 2
os.environ["PYSPARK_PYTHON"] = "/usr/bin/python3.6" 
os.environ["PYSPARK_DRIVER_PYTHON"] = "/usr/bin/python3"
sys.path.insert(0, os.environ["PYLIB"] +"/py4j-0.10.7-src.zip")
sys.path.insert(0, os.environ["PYLIB"] +"/pyspark.zip")

# NOTE: Whichever package you want mention here.
# os.environ['PYSPARK_SUBMIT_ARGS'] = '--packages com.databricks:spark-xml_2.11:0.6.0 pyspark-shell' 
# os.environ['PYSPARK_SUBMIT_ARGS'] = '--packages org.apache.spark:spark-avro_2.11:2.4.0 pyspark-shell'
os.environ['PYSPARK_SUBMIT_ARGS'] = '--packages com.databricks:spark-xml_2.11:0.6.0,org.apache.spark:spark-avro_2.11:2.4.3 pyspark-shell'
# os.environ['PYSPARK_SUBMIT_ARGS'] = '--packages com.databricks:spark-xml_2.11:0.6.0,org.apache.spark:spark-avro_2.11:2.4.0 pyspark-shell'pyspark-shell'

In [3]:
#Entrypoint 2.x
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("Spark SQL basic example").enableHiveSupport().getOrCreate()

# On yarn:
# spark = SparkSession.builder.appName("Spark SQL basic example").enableHiveSupport().master("yarn").getOrCreate()
# specify .master("yarn")

sc = spark.sparkContext

In [9]:
df_csv = spark.read.csv("autos.csv", header=True, inferSchema=True)

In [10]:
df_csv.printSchema()

root
 |-- index: string (nullable = true)
 |-- dateCrawled: string (nullable = true)
 |-- name: string (nullable = true)
 |-- seller: string (nullable = true)
 |-- offerType: string (nullable = true)
 |-- price: string (nullable = true)
 |-- abtest: string (nullable = true)
 |-- vehicleType: string (nullable = true)
 |-- yearOfRegistration: string (nullable = true)
 |-- gearbox: string (nullable = true)
 |-- powerPS: string (nullable = true)
 |-- model: string (nullable = true)
 |-- kilometer: integer (nullable = true)
 |-- monthOfRegistration: string (nullable = true)
 |-- fuelType: string (nullable = true)
 |-- brand: string (nullable = true)
 |-- notRepairedDamage: string (nullable = true)
 |-- dateCreated: string (nullable = true)
 |-- nrOfPictures: integer (nullable = true)
 |-- postalCode: string (nullable = true)
 |-- lastSeen: timestamp (nullable = true)



In [11]:
df_csv.count()

371824

In [12]:
df_csv_duplicates=df_csv.dropDuplicates()
df_csv_duplicates.count()

371824

In [13]:
df_csv.columns


['index',
 'dateCrawled',
 'name',
 'seller',
 'offerType',
 'price',
 'abtest',
 'vehicleType',
 'yearOfRegistration',
 'gearbox',
 'powerPS',
 'model',
 'kilometer',
 'monthOfRegistration',
 'fuelType',
 'brand',
 'notRepairedDamage',
 'dateCreated',
 'nrOfPictures',
 'postalCode',
 'lastSeen']

In [14]:
df_csv.select('price').describe().show()

+-------+------------------+
|summary|             price|
+-------+------------------+
|  count|            371823|
|   mean|17295.124087885943|
| stddev|  3587958.57312194|
|    min|                 0|
|    max|              test|
+-------+------------------+



In [15]:
df_csv.filter("vehicleType is NULL").show()

+-----+-------------------+--------------------+------+---------+-----+-------+-----------+------------------+-------+-------+--------+---------+-------------------+--------+----------+-----------------+-------------------+------------+----------+-------------------+
|index|        dateCrawled|                name|seller|offerType|price| abtest|vehicleType|yearOfRegistration|gearbox|powerPS|   model|kilometer|monthOfRegistration|fuelType|     brand|notRepairedDamage|        dateCreated|nrOfPictures|postalCode|           lastSeen|
+-----+-------------------+--------------------+------+---------+-----+-------+-----------+------------------+-------+-------+--------+---------+-------------------+--------+----------+-----------------+-------------------+------------+----------+-------------------+
|    0|2016-03-24 11:52:17|          Golf_3_1.6|privat|  Angebot|  480|   test|       null|              1993|manuell|      0|    golf|   150000|                  0|  benzin|volkswagen|           

In [16]:
df_csv=df_csv.drop('nrOfPictures','seller','lastSeen','dateCreated')

In [17]:
df_csv.columns

['index',
 'dateCrawled',
 'name',
 'offerType',
 'price',
 'abtest',
 'vehicleType',
 'yearOfRegistration',
 'gearbox',
 'powerPS',
 'model',
 'kilometer',
 'monthOfRegistration',
 'fuelType',
 'brand',
 'notRepairedDamage',
 'postalCode']

In [18]:
df_csv.select('brand').distinct().show()

+-------------+
|        brand|
+-------------+
|       jaguar|
|         nein|
|     daihatsu|
|   mitsubishi|
|         null|
|         lada|
|       toyota|
|         seat|
|         saab|
|   land_rover|
|      peugeot|
|     chrysler|
|      citroen|
|         audi|
|mercedes_benz|
|          bmw|
|         jeep|
|       lancia|
|        skoda|
|        rover|
+-------------+
only showing top 20 rows



In [19]:
from pyspark.sql.functions import to_timestamp

In [20]:
df_csv = df_csv.withColumn('dateCrawled', to_timestamp(df_csv["dateCrawled"], "yyyy-MM-dd HH:mm:ss"))


In [21]:
df_csv.printSchema()

root
 |-- index: string (nullable = true)
 |-- dateCrawled: timestamp (nullable = true)
 |-- name: string (nullable = true)
 |-- offerType: string (nullable = true)
 |-- price: string (nullable = true)
 |-- abtest: string (nullable = true)
 |-- vehicleType: string (nullable = true)
 |-- yearOfRegistration: string (nullable = true)
 |-- gearbox: string (nullable = true)
 |-- powerPS: string (nullable = true)
 |-- model: string (nullable = true)
 |-- kilometer: integer (nullable = true)
 |-- monthOfRegistration: string (nullable = true)
 |-- fuelType: string (nullable = true)
 |-- brand: string (nullable = true)
 |-- notRepairedDamage: string (nullable = true)
 |-- postalCode: string (nullable = true)



In [22]:
df_csv.select('dateCrawled').show(5)

+-------------------+
|        dateCrawled|
+-------------------+
|2016-03-24 11:52:17|
|2016-03-24 10:58:45|
|2016-03-14 12:52:21|
|2016-03-17 16:54:04|
|2016-03-31 17:25:20|
+-------------------+
only showing top 5 rows



In [23]:
from pyspark.sql.functions import to_timestamp, year
df_csv = df_csv.withColumn("dateCrawled", year(df_csv["dateCrawled"]))

In [24]:
df_csv.select('dateCrawled').show(5)

+-----------+
|dateCrawled|
+-----------+
|       2016|
|       2016|
|       2016|
|       2016|
|       2016|
+-----------+
only showing top 5 rows



In [25]:
df_csv.printSchema()

root
 |-- index: string (nullable = true)
 |-- dateCrawled: integer (nullable = true)
 |-- name: string (nullable = true)
 |-- offerType: string (nullable = true)
 |-- price: string (nullable = true)
 |-- abtest: string (nullable = true)
 |-- vehicleType: string (nullable = true)
 |-- yearOfRegistration: string (nullable = true)
 |-- gearbox: string (nullable = true)
 |-- powerPS: string (nullable = true)
 |-- model: string (nullable = true)
 |-- kilometer: integer (nullable = true)
 |-- monthOfRegistration: string (nullable = true)
 |-- fuelType: string (nullable = true)
 |-- brand: string (nullable = true)
 |-- notRepairedDamage: string (nullable = true)
 |-- postalCode: string (nullable = true)



In [26]:
df_csv.select('yearOfRegistration').show(10)

+------------------+
|yearOfRegistration|
+------------------+
|              1993|
|              2011|
|              2004|
|              2001|
|              2008|
|              1995|
|              2004|
|              1980|
|              2014|
|              1998|
+------------------+
only showing top 10 rows



In [27]:
from pyspark.sql.functions import col, when
df_csv = df_csv.withColumn('yearOfRegistration', col('yearOfRegistration').cast("int"))

In [28]:
df_csv.printSchema()

root
 |-- index: string (nullable = true)
 |-- dateCrawled: integer (nullable = true)
 |-- name: string (nullable = true)
 |-- offerType: string (nullable = true)
 |-- price: string (nullable = true)
 |-- abtest: string (nullable = true)
 |-- vehicleType: string (nullable = true)
 |-- yearOfRegistration: integer (nullable = true)
 |-- gearbox: string (nullable = true)
 |-- powerPS: string (nullable = true)
 |-- model: string (nullable = true)
 |-- kilometer: integer (nullable = true)
 |-- monthOfRegistration: string (nullable = true)
 |-- fuelType: string (nullable = true)
 |-- brand: string (nullable = true)
 |-- notRepairedDamage: string (nullable = true)
 |-- postalCode: string (nullable = true)



In [29]:
df_csv = df_csv.withColumn('Price', col('Price').cast("float"))

In [30]:
df_csv.printSchema()

root
 |-- index: string (nullable = true)
 |-- dateCrawled: integer (nullable = true)
 |-- name: string (nullable = true)
 |-- offerType: string (nullable = true)
 |-- Price: float (nullable = true)
 |-- abtest: string (nullable = true)
 |-- vehicleType: string (nullable = true)
 |-- yearOfRegistration: integer (nullable = true)
 |-- gearbox: string (nullable = true)
 |-- powerPS: string (nullable = true)
 |-- model: string (nullable = true)
 |-- kilometer: integer (nullable = true)
 |-- monthOfRegistration: string (nullable = true)
 |-- fuelType: string (nullable = true)
 |-- brand: string (nullable = true)
 |-- notRepairedDamage: string (nullable = true)
 |-- postalCode: string (nullable = true)



In [31]:
df_csv = df_csv.withColumn('monthOfRegistration', col('monthOfRegistration').cast("int"))

In [32]:
df_csv.printSchema()

root
 |-- index: string (nullable = true)
 |-- dateCrawled: integer (nullable = true)
 |-- name: string (nullable = true)
 |-- offerType: string (nullable = true)
 |-- Price: float (nullable = true)
 |-- abtest: string (nullable = true)
 |-- vehicleType: string (nullable = true)
 |-- yearOfRegistration: integer (nullable = true)
 |-- gearbox: string (nullable = true)
 |-- powerPS: string (nullable = true)
 |-- model: string (nullable = true)
 |-- kilometer: integer (nullable = true)
 |-- monthOfRegistration: integer (nullable = true)
 |-- fuelType: string (nullable = true)
 |-- brand: string (nullable = true)
 |-- notRepairedDamage: string (nullable = true)
 |-- postalCode: string (nullable = true)



In [33]:
correlation_value2_value3 = df_csv.stat.corr("monthOfRegistration", "Price")
print(f"Correlation between value2 and value3: {correlation_value2_value3}")

Correlation between value2 and value3: -0.0028176063434703284


In [34]:
correlation_value2_value3 = df_csv.stat.corr("dateCrawled", "Price")
print(f"Correlation between value2 and value3: {correlation_value2_value3}")

Correlation between value2 and value3: 0.00013600453977892399


In [35]:
df_csv.select("monthOfRegistration").distinct().show()

+-------------------+
|monthOfRegistration|
+-------------------+
|                 12|
|               null|
|                  1|
|                  6|
|                  3|
|                  5|
|                  9|
|                  4|
|                  8|
|                  7|
|                 10|
|                 11|
|                  2|
|                  0|
+-------------------+



In [36]:
df_csv=df_csv.drop('offerType')

In [37]:
df_csv.columns

['index',
 'dateCrawled',
 'name',
 'Price',
 'abtest',
 'vehicleType',
 'yearOfRegistration',
 'gearbox',
 'powerPS',
 'model',
 'kilometer',
 'monthOfRegistration',
 'fuelType',
 'brand',
 'notRepairedDamage',
 'postalCode']

In [38]:
zero_count = df_csv.filter(col("Price") == 0).count()
print(zero_count)

10779


In [39]:
df_csv.columns

['index',
 'dateCrawled',
 'name',
 'Price',
 'abtest',
 'vehicleType',
 'yearOfRegistration',
 'gearbox',
 'powerPS',
 'model',
 'kilometer',
 'monthOfRegistration',
 'fuelType',
 'brand',
 'notRepairedDamage',
 'postalCode']

In [40]:
from pyspark.sql.functions import col, sum
df=df_csv
# Get the list of columns
columns = df.columns

# Create a dictionary to store the null count for each column
null_counts = {col_name: df.filter(col(col_name).isNull()).count() for col_name in columns}

# Convert the dictionary to a DataFrame for better display
null_counts_df = spark.createDataFrame([(k, v) for k, v in null_counts.items()], ["Column", "NullCount"])

# Show the null counts
null_counts_df.show()

+-------------------+---------+
|             Column|NullCount|
+-------------------+---------+
|              index|        0|
|        dateCrawled|      296|
|               name|        0|
|              Price|      297|
|             abtest|       30|
|        vehicleType|    37871|
| yearOfRegistration|      297|
|            gearbox|    20211|
|            powerPS|       15|
|              model|    20485|
|          kilometer|        1|
|monthOfRegistration|      297|
|           fuelType|    33388|
|              brand|       64|
|  notRepairedDamage|    72061|
|         postalCode|        1|
+-------------------+---------+



In [41]:
df.select('Price').describe().show()

+-------+------------------+
|summary|             Price|
+-------+------------------+
|  count|            371527|
|   mean|17295.124128259857|
| stddev|3587958.5758335786|
|    min|               0.0|
|    max|      2.14748365E9|
+-------+------------------+



In [42]:
# Calculate the 5th and 95th percentiles
quantiles = df.approxQuantile("price", [0.05, 0.95], 0.01)

# Extract the 5th and 95th percentiles
quantile_05 = quantiles[0]
quantile_95 = quantiles[1]

print(f"5th percentile: {quantile_05}")
print(f"95th percentile: {quantile_95}")

5th percentile: 250.0
95th percentile: 19500.0


In [43]:
df_copy=df_csv

In [44]:
df_copy = df_copy.filter(col("price") != 0)


In [45]:

df=df_copy
# Get the list of columns
columns = df.columns

# Create a dictionary to store the null count for each column
null_counts = {col_name: df.filter(col(col_name).isNull()).count() for col_name in columns}

# Convert the dictionary to a DataFrame for better display
null_counts_df = spark.createDataFrame([(k, v) for k, v in null_counts.items()], ["Column", "NullCount"])

# Show the null counts
null_counts_df.show()

+-------------------+---------+
|             Column|NullCount|
+-------------------+---------+
|              index|        0|
|        dateCrawled|        0|
|               name|        0|
|              Price|        0|
|             abtest|        0|
|        vehicleType|    34127|
| yearOfRegistration|        0|
|            gearbox|    17729|
|            powerPS|        0|
|              model|    18300|
|          kilometer|        0|
|monthOfRegistration|        0|
|           fuelType|    29948|
|              brand|        0|
|  notRepairedDamage|    66769|
|         postalCode|        0|
+-------------------+---------+



In [46]:
most_common_row = df_copy.groupBy("vehicleType").count().orderBy(col("count").desc()).first()
most_common = most_common_row["vehicleType"]
df_VehicleCopy = df_copy.fillna({"vehicleType": most_common})

In [47]:
df=df_VehicleCopy
# Get the list of columns
columns = df.columns

# Create a dictionary to store the null count for each column
null_counts = {col_name: df.filter(col(col_name).isNull()).count() for col_name in columns}

# Convert the dictionary to a DataFrame for better display
null_counts_df = spark.createDataFrame([(k, v) for k, v in null_counts.items()], ["Column", "NullCount"])

# Show the null counts
null_counts_df.show()

+-------------------+---------+
|             Column|NullCount|
+-------------------+---------+
|              index|        0|
|        dateCrawled|        0|
|               name|        0|
|              Price|        0|
|             abtest|        0|
|        vehicleType|        0|
| yearOfRegistration|        0|
|            gearbox|    17729|
|            powerPS|        0|
|              model|    18300|
|          kilometer|        0|
|monthOfRegistration|        0|
|           fuelType|    29948|
|              brand|        0|
|  notRepairedDamage|    66769|
|         postalCode|        0|
+-------------------+---------+



In [48]:
value_counts = df_VehicleCopy.groupBy("vehicleType").count()

# Step 2: Sort by count in descending order
sorted_value_counts = value_counts.orderBy(col("count").desc())
sorted_value_counts.show()

+-----------+------+
|vehicleType| count|
+-----------+------+
|  limousine|127969|
| kleinwagen| 78209|
|      kombi| 66098|
|        bus| 29769|
|     cabrio| 22560|
|      coupe| 18487|
|        suv| 14501|
|     andere|  3155|
+-----------+------+



In [49]:
most_common_row =df_VehicleCopy.groupBy("gearbox").count().orderBy(col("count").desc()).first()
most_common = most_common_row["gearbox"]

# Step 2: Replace missing values in 'gearbox' column with the most common value
df_VehicleCopy = df_VehicleCopy.fillna({"gearbox": most_common})

In [50]:
df=df_VehicleCopy
# Get the list of columns
columns = df.columns

# Create a dictionary to store the null count for each column
null_counts = {col_name: df.filter(col(col_name).isNull()).count() for col_name in columns}

# Convert the dictionary to a DataFrame for better display
null_counts_df = spark.createDataFrame([(k, v) for k, v in null_counts.items()], ["Column", "NullCount"])

# Show the null counts
null_counts_df.show()

+-------------------+---------+
|             Column|NullCount|
+-------------------+---------+
|              index|        0|
|        dateCrawled|        0|
|               name|        0|
|              Price|        0|
|             abtest|        0|
|        vehicleType|        0|
| yearOfRegistration|        0|
|            gearbox|        0|
|            powerPS|        0|
|              model|    18300|
|          kilometer|        0|
|monthOfRegistration|        0|
|           fuelType|    29948|
|              brand|        0|
|  notRepairedDamage|    66769|
|         postalCode|        0|
+-------------------+---------+



In [51]:
most_common_row =df_VehicleCopy.groupBy("fuelType").count().orderBy(col("count").desc()).first()
most_common = most_common_row["fuelType"]

# Step 2: Replace missing values in 'gearbox' column with the most common value
df_VehicleCopy = df_VehicleCopy.fillna({"fuelType": most_common})

In [52]:
df=df_VehicleCopy
# Get the list of columns
columns = df.columns

# Create a dictionary to store the null count for each column
null_counts = {col_name: df.filter(col(col_name).isNull()).count() for col_name in columns}

# Convert the dictionary to a DataFrame for better display
null_counts_df = spark.createDataFrame([(k, v) for k, v in null_counts.items()], ["Column", "NullCount"])

# Show the null counts
null_counts_df.show()

+-------------------+---------+
|             Column|NullCount|
+-------------------+---------+
|              index|        0|
|        dateCrawled|        0|
|               name|        0|
|              Price|        0|
|             abtest|        0|
|        vehicleType|        0|
| yearOfRegistration|        0|
|            gearbox|        0|
|            powerPS|        0|
|              model|    18300|
|          kilometer|        0|
|monthOfRegistration|        0|
|           fuelType|        0|
|              brand|        0|
|  notRepairedDamage|    66769|
|         postalCode|        0|
+-------------------+---------+



In [53]:
most_common_row =df_VehicleCopy.groupBy("notRepairedDamage").count().orderBy(col("count").desc()).first()
most_common = most_common_row["notRepairedDamage"]

# Step 2: Replace missing values in 'gearbox' column with the most common value
df_VehicleCopy = df_VehicleCopy.fillna({"notRepairedDamage": most_common})

In [71]:
df=df_VehicleCopy
# Get the list of columns
columns = df.columns

# Create a dictionary to store the null count for each column
null_counts = {col_name: df.filter(col(col_name).isNull()).count() for col_name in columns}

# Convert the dictionary to a DataFrame for better display
null_counts_df = spark.createDataFrame([(k, v) for k, v in null_counts.items()], ["Column", "NullCount"])

# Show the null counts
null_counts_df.show()

+-------------------+---------+
|             Column|NullCount|
+-------------------+---------+
|              index|        0|
|        dateCrawled|        0|
|               name|        0|
|              Price|        0|
|             abtest|        0|
|        vehicleType|        0|
| yearOfRegistration|        0|
|            gearbox|        0|
|            powerPS|        0|
|              model|    18300|
|          kilometer|        0|
|monthOfRegistration|        0|
|           fuelType|        0|
|              brand|        0|
|  notRepairedDamage|        0|
|         postalCode|        0|
+-------------------+---------+



In [72]:
df_VehicleCopy.columns

['index',
 'dateCrawled',
 'name',
 'Price',
 'abtest',
 'vehicleType',
 'yearOfRegistration',
 'gearbox',
 'powerPS',
 'model',
 'kilometer',
 'monthOfRegistration',
 'fuelType',
 'brand',
 'notRepairedDamage',
 'postalCode']

In [73]:
df_day2=df_VehicleCopy

In [74]:
df=df_day2

In [77]:
from pyspark.sql.functions import col, sum
df.select('powerPS').describe().show()

+-------+------------------+
|summary|           powerPS|
+-------+------------------+
|  count|            360748|
|   mean|116.57623604288867|
| stddev|190.60860977764153|
|    min|                 0|
|    max|             20000|
+-------+------------------+



In [76]:
df=df.withColumn("powerPS", col("powerPS").cast("int"))

In [78]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col

# Calculate the 5th and 95th percentiles
quantiles = df.approxQuantile("powerPS", [0.05, 0.95], 0.01)

# Extract the 5th and 95th percentiles
quantile_05 = quantiles[0]
quantile_95 = quantiles[1]

print(f"5th percentile: {quantile_05}")
print(f"95th percentile: {quantile_95}")

5th percentile: 0.0
95th percentile: 231.0


In [79]:
df_imputed = df.withColumn("powerPS", 
                           when(col("powerPS") < 40, 40)
                           .when(col("powerPS") > 15000, 15000)
                           .otherwise(col("powerPS")))
df_imputed.show(5)

+-----+-----------+--------------------+-------+------+-----------+------------------+---------+-------+-----+---------+-------------------+--------+----------+-----------------+----------+
|index|dateCrawled|                name|  Price|abtest|vehicleType|yearOfRegistration|  gearbox|powerPS|model|kilometer|monthOfRegistration|fuelType|     brand|notRepairedDamage|postalCode|
+-----+-----------+--------------------+-------+------+-----------+------------------+---------+-------+-----+---------+-------------------+--------+----------+-----------------+----------+
|    0|       2016|          Golf_3_1.6|  480.0|  test|  limousine|              1993|  manuell|     40| golf|   150000|                  0|  benzin|volkswagen|             nein|     70435|
|    1|       2016|A5_Sportback_2.7_Tdi|18300.0|  test|      coupe|              2011|  manuell|    190| null|   125000|                  5|  diesel|      audi|               ja|     66954|
|    2|       2016|"Jeep_Grand_Chero...| 9800.0|  

In [80]:
df=df_imputed

In [85]:
df.select('model').distinct().count()

252

In [86]:
df_VehicleCopy=df
most_common_row =df_VehicleCopy.groupBy("model").count().orderBy(col("count").desc()).first()
most_common = most_common_row["model"]

# Step 2: Replace missing values in 'gearbox' column with the most common value
df_VehicleCopy = df_VehicleCopy.fillna({"model": most_common})

In [87]:
df=df_VehicleCopy

In [88]:

# Get the list of columns
columns = df.columns

# Create a dictionary to store the null count for each column
null_counts = {col_name: df.filter(col(col_name).isNull()).count() for col_name in columns}

# Convert the dictionary to a DataFrame for better display
null_counts_df = spark.createDataFrame([(k, v) for k, v in null_counts.items()], ["Column", "NullCount"])

# Show the null counts
null_counts_df.show()

+-------------------+---------+
|             Column|NullCount|
+-------------------+---------+
|              index|        0|
|        dateCrawled|        0|
|               name|        0|
|              Price|        0|
|             abtest|        0|
|        vehicleType|        0|
| yearOfRegistration|        0|
|            gearbox|        0|
|            powerPS|        0|
|              model|        0|
|          kilometer|        0|
|monthOfRegistration|        0|
|           fuelType|        0|
|              brand|        0|
|  notRepairedDamage|        0|
|         postalCode|        0|
+-------------------+---------+



In [89]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import when, col
df = df.withColumn("monthOfRegistration", when(col("monthOfRegistration") == 0, 1).otherwise(col("monthOfRegistration")))

In [90]:
zero_count = df.filter(col("monthOfRegistration") == 0).count()
print(zero_count)

0


In [91]:
df = df.withColumn("postalCode", col("postalCode").cast("int"))

In [92]:
df.select('postalCode').distinct().count()

8146

In [94]:
df.select('gearbox').distinct().show()

+---------+
|  gearbox|
+---------+
|automatik|
|  manuell|
+---------+



In [95]:
# Rename column values
df_renamed = df.withColumn('gearbox', 
                           when(col('gearbox') == "automatik", "Automatic")
                           .when(col('gearbox') == "manuell", "Manual")
                          )
df_renamed.show(5)

+-----+-----------+--------------------+-------+------+-----------+------------------+---------+-------+-----+---------+-------------------+--------+----------+-----------------+----------+
|index|dateCrawled|                name|  Price|abtest|vehicleType|yearOfRegistration|  gearbox|powerPS|model|kilometer|monthOfRegistration|fuelType|     brand|notRepairedDamage|postalCode|
+-----+-----------+--------------------+-------+------+-----------+------------------+---------+-------+-----+---------+-------------------+--------+----------+-----------------+----------+
|    0|       2016|          Golf_3_1.6|  480.0|  test|  limousine|              1993|   Manual|     40| golf|   150000|                  1|  benzin|volkswagen|             nein|     70435|
|    1|       2016|A5_Sportback_2.7_Tdi|18300.0|  test|      coupe|              2011|   Manual|    190| golf|   125000|                  5|  diesel|      audi|               ja|     66954|
|    2|       2016|"Jeep_Grand_Chero...| 9800.0|  

In [97]:
df=df_renamed
df.select('gearbox').distinct().show()

+---------+
|  gearbox|
+---------+
|Automatic|
|   Manual|
+---------+



In [98]:
df.select('vehicleType').distinct().show()

+-----------+
|vehicleType|
+-----------+
|      coupe|
| kleinwagen|
|        bus|
|     andere|
|  limousine|
|     cabrio|
|        suv|
|      kombi|
+-----------+



In [100]:
df_renamed = df.withColumn('vehicleType', 
                           when(col('vehicleType') == "coupe", "coupe")
                           .when(col('vehicleType') == "kleinwagen", "small_car")
                           .when(col( 'vehicleType' ) == "limousine", "sedan")
                           .when(col('vehicleType') == "cabrio", "convertible")
                           .when(col('vehicleType') == "bus", "bus")
                           .when(col('vehicleType') == "kombi", "station_wagon")
                           .when(col('vehicleType') == "andere", "Others")
                           .when(col('vehicleType') == "suv", "suv")
                          )

                          
    

In [101]:
df=df_renamed
df.select('vehicleType').distinct().show()

+-------------+
|  vehicleType|
+-------------+
|        coupe|
|          bus|
|station_wagon|
|    small_car|
|          suv|
|  convertible|
|       Others|
|        sedan|
+-------------+



In [104]:
df.select('fuelType').distinct().show()

+--------+
|fuelType|
+--------+
|  benzin|
| elektro|
|  andere|
|  diesel|
|     cng|
|  hybrid|
|     lpg|
+--------+



In [105]:
df_renamed = df.withColumn('fuelType', 
                           when(col('fuelType') == "benzin", "Petrol")
                           .when(col('fuelType') == "elektro", "Electric")
                           .when(col( 'fuelType' ) == "lpg", "LPG")
                           .when(col('fuelType') == "diesel", "Diesel")
                           .when(col('fuelType') == "cng", "CNG")
                           .when(col('fuelType') == "hybrid", "Hybrid")
                           .when(col('fuelType') == "andere", "Others")
                          )

In [107]:
df=df_renamed
df.select('fuelType').distinct().show()

+--------+
|fuelType|
+--------+
|  Diesel|
|  Hybrid|
|     CNG|
|  Others|
|Electric|
|     LPG|
|  Petrol|
+--------+



In [108]:
#df = spark.createDataFrame(df_VehicleCopy, df_VehicleCopy.columns)
df_VehicleCopy=df
pandas_df = df_VehicleCopy.toPandas()

In [109]:
df_VehicleCopy.show(5)

+-----+-----------+--------------------+-------+------+-----------+------------------+---------+-------+-----+---------+-------------------+--------+----------+-----------------+----------+
|index|dateCrawled|                name|  Price|abtest|vehicleType|yearOfRegistration|  gearbox|powerPS|model|kilometer|monthOfRegistration|fuelType|     brand|notRepairedDamage|postalCode|
+-----+-----------+--------------------+-------+------+-----------+------------------+---------+-------+-----+---------+-------------------+--------+----------+-----------------+----------+
|    0|       2016|          Golf_3_1.6|  480.0|  test|      sedan|              1993|   Manual|     40| golf|   150000|                  1|  Petrol|volkswagen|             nein|     70435|
|    1|       2016|A5_Sportback_2.7_Tdi|18300.0|  test|      coupe|              2011|   Manual|    190| golf|   125000|                  5|  Diesel|      audi|               ja|     66954|
|    2|       2016|"Jeep_Grand_Chero...| 9800.0|  

In [112]:
output_path = "day2_data.csv"  # Change this to your desired output path

# Save DataFrame to CSV
df_VehicleCopy.write.csv(output_path, header=True)

In [127]:
pip install openpyxl


    100% |████████████████████████████████| 256kB 391kB/s ta 0:00:01
Note: you may need to restart the kernel to use updated packages.


In [128]:
import openpyxl


In [129]:
file_path = "/home/talentum/shared/Project_Data_Cleaned/output.xlsx"

# Write Pandas DataFrame to an Excel file
pandas_df.to_excel(file_path, index=False,engine='openpyxl')


In [5]:
df_csv_day2 = spark.read.csv("autos.xlsx", header=True, inferSchema=True)

In [6]:
df=df_csv_day2

In [7]:
df.select('powerps').describe()

AnalysisException: 'cannot resolve \'`powerps`\' given input columns: [PK\x03\x04\x14\x00\x06\x00\x08\x00\x00\x00!\x00b�h^\x01\x00\x00�\x04\x00\x00\x13\x00\x08\x02[Content_Types].xml �\x04\x02(�\x00\x02\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x